In [1]:
import os
import pickle

import numpy as np
import numpy.ma as ma

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site

from models import (
    NonHierarchicalBayesianModel
)

In [2]:
src = "/home/andres/repos/rat-mapping-paper/reports/C_SMA_LAR/non_hierarchical_bayesian_model/processed_inference.pkl"
with open(src, "rb") as f:
    (
        df,
        encoder_dict,
        model,
        posterior_samples,
        y,
        subjects_inv,
        compound_positions_inv,
        compound_size_inv,
    ) = pickle.load(f)


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [19]:
a = posterior_samples[site.a]
a.shape


(400, 8, 70, 3, 6)

In [18]:
y.shape # samples, subjects, positions, sizes, muscles, x) 

(400, 8, 70, 3, 6, 1000)

In [8]:
y_swap = ma.swapaxes(y, 0, 1)
y_swap = ma.swapaxes(y_swap, 1, 2)
y_swap.shape

(8, 70, 400, 3, 6, 1000)

In [3]:
t = list(zip(subjects_inv, compound_positions_inv, compound_size_inv))


In [7]:
compound_size_inv

['B', 'S', 'S-B']

In [13]:
y_temp= y[:, :, :, 0, ...]

In [14]:
temp_mean = ma.mean(y_temp[:, 1, ...], axis = 1)

In [17]:
temp_mean.shape

(400, 6, 1000)

In [9]:
compound_positions_inv

['-C5L',
 '-C5LL',
 '-C5LM',
 '-C5LM1',
 '-C5LM2',
 '-C5M',
 '-C6L',
 '-C6LL',
 '-C6LM',
 '-C6LM1',
 '-C6LM2',
 '-C6M',
 '-C6R',
 '-C6RM',
 '-C6RR',
 '-C7LM',
 '-C7M',
 '-C7R',
 '-C7RM',
 '-C7RR',
 'C5L-C5LL',
 'C5LM1-C5L',
 'C5LM1-C5LL',
 'C5LM2-C5L',
 'C5LM2-C5LL',
 'C5LM2-C5LM1',
 'C5LM2-C5M',
 'C5LM2-C6L',
 'C5LM2-C6LL',
 'C5M-C5L',
 'C5M-C5LL',
 'C5M-C5LM',
 'C5M-C5LM1',
 'C5M-C5LM2',
 'C5M-C6L',
 'C5M-C6LL',
 'C6L-C6LL',
 'C6LL-C6L',
 'C6LM-C6L',
 'C6LM-C6LL',
 'C6LM-C6M',
 'C6LM-C6R',
 'C6LM-C6RM',
 'C6LM-C6RR',
 'C6LM1-C6L',
 'C6LM1-C6LL',
 'C6LM2-C6L',
 'C6LM2-C6LL',
 'C6LM2-C6LM1',
 'C6M-C6L',
 'C6M-C6LL',
 'C6M-C6LM',
 'C6M-C6LM1',
 'C6M-C6LM2',
 'C6M-C6R',
 'C6M-C6RM',
 'C6M-C6RR',
 'C6R-C6RR',
 'C6RM-C6R',
 'C6RM-C6RR',
 'C7LM-C7M',
 'C7LM-C7R',
 'C7LM-C7RM',
 'C7LM-C7RR',
 'C7M-C7R',
 'C7M-C7RM',
 'C7M-C7RR',
 'C7R-C7RR',
 'C7RM-C7R',
 'C7RM-C7RR']

In [ ]:
#check for masks? 

def get_entropy(subset):
    y = y[:, subset, ...]
    x = y[:, subset, 0, 0, :]

    y_max = ma.max(y, axis=(1, -1), keepdims=True) 
    y_max.shape

    y = ma.where(y, y / y_max, 0.)
    y.shape

    p = ma.sum(y, axis=-2, keepdims=True)
    p = ma.where(y, y / p, 0.)
    p.shape

    plogp = ma.where(p, p * ma.log(p), 0.)
    plogp.shape

    entropy = ma.where(
        ma.any(p, axis=-2, keepdims=True),
        (
            1
            + (ma.sum(plogp, axis=-2, keepdims=True) / np.log(plogp.shape[-2]))
        ),
        0.
    )
    entropy = entropy[..., 0, :]
    entropy.shape

    return entropy, x
